In [ ]:
################# 실제 작동되는 코드 ####################

import paramiko
import os

ip = "192.168.0.17"
user = "root"
pw = "asd123!@"
ZONE_FILE_NAME = ""
ZONE_NAME = ""

#######################
# 공용 SSH / SFTP 함수
#######################
def get_sftp():
    transport = paramiko.Transport((ip, 22))
    transport.connect(username=user, password=pw)
    sftp = paramiko.SFTPClient.from_transport(transport)
    return sftp, transport

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err

def r_ssh(remote_path):
    sftp, transport = get_sftp()
    with sftp.open(remote_path, "r") as f:
        data = f.read().decode()
    sftp.close()
    transport.close()
    return data

def w_ssh(remote_path, content):
    sftp, transport = get_sftp()
    with sftp.open(remote_path, "w") as f:
        f.write(content)
    sftp.close()
    transport.close()

def a_ssh(remote_path, content):
    sftp, transport = get_sftp()
    with sftp.open(remote_path, "a") as f:
        f.write(content)
    sftp.close()
    transport.close()

def pp(output):
    for i in output.splitlines():
        print(i.strip())


#############################
#     기능 함수 영역         #
#############################
def install():
    print("=== BIND9 설치 ===")
    pp(cc("apt-get update -y"))
    pp(cc("apt-get install -y bind9 bind9-utils bind9-dnsutils"))
    pp(cc("systemctl status bind9"))

def mk_zone_file():
    global ZONE_FILE_NAME
    ZONE_FILE_NAME = input("생성할 zone 파일을 입력해주세요 (ex. test.com.zone): ")
    print(f"db.local(example) 파일을 복사해 {ZONE_FILE_NAME} zone 파일을 생성합니다.")
    cc(f"cp -ap /etc/bind/db.local /etc/bind/{ZONE_FILE_NAME}")
    print(f"✅ zone 파일이 생성되었습니다: /etc/bind/{ZONE_FILE_NAME}")

def set_zone_file():
    global ZONE_FILE_NAME
    print("zone file의 내용을 수정하겠습니다.")
    domain = input("사용할 도메인 주소 (ex. ns.test.com): ")
    ipaddr = input("현재 사용하고 있는 DNS의 ip (ip a 확인) (ex. 172.16.16.100): ")
    contents = f"""$TTL    604800
@       IN      SOA     {domain}. root.adminemail.com. (
                              2         ; Serial
                         604800         ; Refresh
                          86400         ; Retry
                        2419200         ; Expire
                         604800 )       ; Negative Cache TTL
;
@       IN      NS      {domain}.
ns       IN      A       {ipaddr}
@       IN      AAAA    ::1
"""
    w_ssh(f"/etc/bind/{ZONE_FILE_NAME}", contents)
    print("✅ zone file 내용 수정 완료")

def set_named_conf_options():
    print("bind9 설정 파일을 수정하겠습니다. (경로 : /etc/bind/named.conf.options)")
    contents = """options {
        directory "/var/cache/bind";
        dnssec-validation auto;
        recursion yes;
        allow-query { any; };
        listen-on { any; };
        listen-on-v6 { any; };
};
"""
    conf_path = "/etc/bind/named.conf.options"
    cc(f"cp -ap {conf_path} {conf_path}.bak")
    w_ssh(conf_path, contents)
    print("✅ named.conf.options 수정 완료")

def set_named_conf_local():
    global ZONE_NAME, ZONE_FILE_NAME
    print("bind9 설정 파일을 수정하겠습니다. (경로 : /etc/bind/named.conf.local)")
    ZONE_NAME = input("zone의 이름을 입력해주세요 (ex. test.com): ")
    contents = f"""
zone "{ZONE_NAME}" {{
    type master;
    file "/etc/bind/{ZONE_FILE_NAME}";
}};
"""
    a_ssh("/etc/bind/named.conf.local", contents)
    print(f"✅ zone '{ZONE_NAME}' 추가 완료 (/etc/bind/named.conf.local)")

def start_dns():
    pp(cc("systemctl restart bind9"))
    pp(cc("systemctl status bind9"))

#############################
#  dns_data를 활용해 dns관리 #
#############################

def create_dns_file():
    file_name = "dns_data.txt"
    target_path = "/root/jupyter/security/output"
    cc(f"touch {target_path}{file_name}")
    print(f"✅ {target_path}{file_name} 생성 완료")

def info():
    name = input("이름을 입력하세요: ")
    domain_id = input("사용할 도메인 이름을 입력하세요: ")
    ip = input("IP를 입력하세요: ")
    data = f"{name} {domain_id} {ip}\n"
    return data

def dns_mod():
    file_path = "/root/jupyter/security/output/dns_data.txt"
    while True:
        lines = r_ssh(file_path)
        lines_list = lines.splitlines()

        print("\n==== 현재 파일 내용 ====")
        for i, line in enumerate(lines_list, 1):
            print(f"{i}: {line}")
        print("======================\n")

        input_num = input("수정(1), 삭제(2), 추가(3), 종료(0): ")

        if input_num == "0":
            print("프로그램 종료")
            break
        elif input_num == "1":
            line_num = int(input("수정할 줄 번호를 입력하세요: "))
            if 1 <= line_num <= len(lines_list):
                new_text = info()
                lines_list[line_num - 1] = new_text
                w_ssh(file_path, "\n".join(lines_list))
        elif input_num == "2":
            line_num = int(input("삭제할 줄 번호를 입력하세요: "))
            if 1 <= line_num <= len(lines_list):
                lines_list.pop(line_num - 1)
                w_ssh(file_path, "\n".join(lines_list))
        elif input_num == "3":
            new_text = info()
            a_ssh(file_path, new_text + "\n")
        else:
            print("잘못된 입력입니다.")


def overwrite_dns():
 #   zone_file = f"/etc/bind/{ZONE_FILE_NAME}"
    zone_file = "/etc/bind/test.com.zone"
    data_file = "/root/jupyter/security/output/dns_data.txt"

    data_lines = r_ssh(data_file).splitlines()
    zone_lines = r_ssh(zone_file).splitlines()

    header = zone_lines[:12]
    new_records = []
    
    for line in data_lines:
        parts = line.strip().split()
        if len(parts) >= 3:
            name, domain, ip = parts[:3]
            new_records.append(f"{domain}\tIN\tA\t{ip}")

    new_zone_content = header + new_records
    new_zone_content_str = "\n".join(new_zone_content) + "\n"

    print(new_zone_content_str)
    a_ssh(zone_file, new_zone_content_str)
    print("Zone 파일이 성공적으로 갱신되었습니다.")


def init():
#    install()
    mk_zone_file()
    ##############
    set_zone_file()
    set_named_conf_options()
    set_named_conf_local()
    start_dns()

def mod_zone():
    create_dns_file
    dns_mod()
    overwrite_dns()

#init()
mod_zone()